In [2]:
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta
import re

In [3]:


# Mocking the content_fetcher tool and SourceReference for self-contained code.
# In a real environment, you would use the actual content_fetcher.
def fetch_mock_t6(query, source_references):
    if source_references[0].id == "uploaded:telegram_preprocessed_data.csv":
        # This is placeholder content mirroring the structure for demonstration.
        # In a real scenario, this would be the actual content from your uploaded CSV.
        return """chat_id,Message,Views,Date
1001,አዲስ አበባ ላይ ጥሩ ጥራት ያለው የህፃናት አልጋ በ2500 ብር ብቻ።,500,2024-01-01 10:00:00
1001,የሴቶች ቦርሳ በ1200 ብር ቦሌ አካባቢ ይገኛል።,750,2024-01-02 11:30:00
1002,ሳሪስ ለገበያ የቀረበ ዘመናዊ ቲቪ 8000 ብር።,1200,2024-01-01 12:00:00
1001,ለቤትዎ ውበት የሆኑ መጋረጃዎች በ650 ብር ከፒያሳ።,600,2024-01-03 09:00:00
1002,የወንዶች ሸሚዝ በ950 ብር አዲስ አበባ።,900,2024-01-02 14:00:00
1001,ጥሩ ስልክ በ1000 ብር ቦሌ ላይ።,1500,2024-01-04 16:00:00
1002,ላፕቶፕ በ15000 ብር ሳሪስ አካባቢ።,1800,2024-01-03 10:00:00
1001,ለልጆች መጫወቻዎች በ300 ብር ከፒያሳ።,400,2024-01-05 08:00:00
1002,የተለያዩ የቤት እቃዎች 7000 ብር አዲስ አበባ።,1000,2024-01-04 13:00:00
1001,ፋሽን ልብሶች 800 ብር ቦሌ።,1100,2024-01-06 17:00:00
1002,የህፃናት መጫወቻዎች በ450 ብር ሳሪስ።,600,2024-01-05 09:00:00
"""
    return ""

class SourceReference:
    def __init__(self, id, type=None):
        self.id = id
        self.type = type

# Fetch the CSV content
csv_content_t6 = fetch_mock_t6(query="telegram_preprocessed_data.csv content", source_references=[SourceReference(id="uploaded:telegram_preprocessed_data.csv")])

# Read the CSV content into a pandas DataFrame
df_task6 = pd.read_csv(StringIO(csv_content_t6))

# Ensure 'Date' column is datetime
df_task6['Date'] = pd.to_datetime(df_task6['Date'])

# --- Mock NER Prediction Function (since we don't have a live model here) ---
# This function simulates the output of an NER model. In a real scenario,
# you would load and use your fine-tuned model from Task 3/4.
def mock_ner_predict(text):
    # This is a very simplified rule-based extraction for demonstration.
    # In reality, this would be the output of your fine-tuned NER model.
    product = "Unknown Product"
    price = None
    location = "Unknown Location"

    # Price extraction: look for numbers followed by "ብር" or "ETB", or "በ" followed by number
    price_match = re.search(r"(\d[\d,.]*)\s*(ብር|ETB)|በ\s*(\d[\d,.]*)\s*ብር", text)
    if price_match:
        if price_match.group(1): # Case: "number ብር" or "number ETB"
            price = float(price_match.group(1).replace(",", ""))
        elif price_match.group(3): # Case: "በ number ብር"
            price = float(price_match.group(3).replace(",", ""))

    # Location extraction: keywords
    for loc_kw in ["አዲስ አበባ", "ቦሌ", "ፒያሳ", "ሳሪስ"]:
        if loc_kw in text:
            location = loc_kw
            break

    # Product extraction (very basic, just picking first noun-like word if no specific product rule)
    product_keywords = {
        "አልጋ": "የህፃናት አልጋ", "ቦርሳ": "የሴቶች ቦርሳ", "ቲቪ": "ዘመናዊ ቲቪ",
        "መጋረጃዎች": "መጋረጃዎች", "ሸሚዝ": "የወንዶች ሸሚዝ", "ስልክ": "ስልክ",
        "ላፕቶፕ": "ላፕቶፕ", "መጫወቻዎች": "መጫወቻዎች", "እቃዎች": "የቤት እቃዎች",
        "ልብሶች": "ፋሽን ልብሶች"
    }
    for kw, prod_name in product_keywords.items():
        if kw in text:
            product = prod_name
            break

    return {
        "product": product,
        "price": price,
        "location": location
    }

# Apply mock NER prediction to each message
df_task6['ner_results'] = df_task6['Message'].apply(mock_ner_predict)

# Extract price and product for easier calculation
df_task6['Extracted_Price'] = df_task6['ner_results'].apply(lambda x: x['price'])
df_task6['Extracted_Product'] = df_task6['ner_results'].apply(lambda x: x['product'])

# --- Develop Vendor Analytics Engine ---
def calculate_vendor_metrics(df_vendor):
    metrics = {}

    # Activity & Consistency: Posting Frequency
    if not df_vendor.empty:
        min_date = df_vendor['Date'].min()
        max_date = df_vendor['Date'].max()
        time_span_days = (max_date - min_date).days
        if time_span_days == 0: # Handle cases with only one post or posts on the same day
            posting_frequency = df_vendor.shape[0] # Number of posts if within one day
        else:
            posting_frequency = (df_vendor.shape[0] / time_span_days) * 7 # Posts per week
    else:
        posting_frequency = 0

    metrics['Posting Frequency (Posts/Week)'] = posting_frequency

    # Market Reach & Engagement: Average Views per Post
    metrics['Average Views per Post'] = df_vendor['Views'].mean() if not df_vendor.empty else 0

    # Top Performing Post
    if not df_vendor.empty:
        top_post = df_vendor.loc[df_vendor['Views'].idxmax()]
        metrics['Top Performing Post Product'] = top_post['Extracted_Product']
        metrics['Top Performing Post Price'] = top_post['Extracted_Price']
        metrics['Top Performing Post Views'] = top_post['Views']
    else:
        metrics['Top Performing Post Product'] = None
        metrics['Top Performing Post Price'] = None
        metrics['Top Performing Post Views'] = 0

    # Business Profile: Average Price Point
    prices = df_vendor['Extracted_Price'].dropna()
    metrics['Average Price Point (ETB)'] = prices.mean() if not prices.empty else 0

    return metrics

# Group by chat_id (vendor channel) and calculate metrics for each
vendor_scores = {}
for chat_id, group in df_task6.groupby('chat_id'):
    vendor_scores[chat_id] = calculate_vendor_metrics(group)

# Create a Final "Lending Score"
# Example formula: Score = (Avg Views * 0.5) + (Posting Frequency * 0.5)
# Let's add Average Price Point with a negative weight (higher price, potentially lower score if not balanced by volume)
# Or, assume higher price point indicates higher value items, so positive weight.
# Let's make it simple: Normalized Avg Views + Normalized Posting Frequency + Normalized Avg Price
def calculate_lending_score(metrics):
    # To normalize correctly, we should have global min/max for each metric from the entire dataset.
    # For this small mock dataset, we'll use arbitrary normalization factors.
    # In a real application, calculate these from your full dataset before applying.

    # Arbitrary maximums for normalization based on mock data's scale
    max_views = 2000.0 # From mock data, max is 1800
    max_posting_freq = 7.0 # If daily posts, up to 7 posts/week, based on mock data span
    max_avg_price = 15000.0 # From mock data, max avg price around 15000

    avg_views_norm = metrics['Average Views per Post'] / max_views
    posting_freq_norm = metrics['Posting Frequency (Posts/Week)'] / max_posting_freq
    avg_price_norm = metrics['Average Price Point (ETB)'] / max_avg_price

    # Ensure normalized values are not negative (e.g., if original value is 0)
    avg_views_norm = max(0, avg_views_norm)
    posting_freq_norm = max(0, posting_freq_norm)
    avg_price_norm = max(0, avg_price_norm)

    # Weights can be adjusted based on business strategy
    # More weight on engagement and activity, less on price point
    lending_score = (avg_views_norm * 0.4) + (posting_freq_norm * 0.4) + (avg_price_norm * 0.2)
    return round(lending_score * 100, 2) # Scale to 0-100 for readability

# Add lending score to vendor_scores
for chat_id, metrics in vendor_scores.items():
    vendor_scores[chat_id]['Lending Score'] = calculate_lending_score(metrics)

# Create a summary table
summary_df = pd.DataFrame.from_dict(vendor_scores, orient='index')
summary_df.index.name = 'Vendor ID (chat_id)'
summary_df_display = summary_df[[
    'Average Views per Post',
    'Posting Frequency (Posts/Week)',
    'Average Price Point (ETB)',
    'Lending Score'
]].round(2)

print("\n--- Vendor Scorecard Summary ---")
print(summary_df_display.to_markdown(index=True))

print("\n--- Top Performing Posts by Vendor ---")
top_posts_df = summary_df[[
    'Top Performing Post Product',
    'Top Performing Post Price',
    'Top Performing Post Views'
]]
print(top_posts_df.to_markdown(index=True))


--- Vendor Scorecard Summary ---
|   Vendor ID (chat_id) |   Average Views per Post |   Posting Frequency (Posts/Week) |   Average Price Point (ETB) |   Lending Score |
|----------------------:|-------------------------:|---------------------------------:|----------------------------:|----------------:|
|                  1001 |                   808.33 |                             8.4  |                        1075 |           65.6  |
|                  1002 |                  1100    |                            11.67 |                        6280 |           97.04 |

--- Top Performing Posts by Vendor ---
|   Vendor ID (chat_id) | Top Performing Post Product   |   Top Performing Post Price |   Top Performing Post Views |
|----------------------:|:------------------------------|----------------------------:|----------------------------:|
|                  1001 | ስልክ                           |                        1000 |                        1500 |
|                  1002 | ላፕ


Using mock NER prediction function for entity extraction (actual NER model not loaded).

--- Vendor Scorecard Summary ---
|   Vendor ID (chat_id) |   Average Views per Post |   Posting Frequency (Posts/Week) |   Average Price Point (ETB) |   Lending Score |
|----------------------:|-------------------------:|---------------------------------:|----------------------------:|----------------:|
|                  1001 |                   808.33 |                             8.4  |                        1075 |               0 |
|                  1002 |                  1100    |                            11.67 |                        6280 |             100 |

--- Top Performing Posts by Vendor ---
|   Vendor ID (chat_id) | Top Performing Post Product   |   Top Performing Post Price |   Top Performing Post Views |
|----------------------:|:------------------------------|----------------------------:|----------------------------:|
|                  1001 | ስልክ                           |